In [295]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from pandas import to_datetime, DataFrame, concat
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0',
    'X-Requested-With': 'XMLHttpRequest',
}

## Общие новости: Интерфакс 

In [296]:
from datetime import date, timedelta

start_date, end_date = date(2022, 9, 27), date(2022, 10, 8)
delta = end_date - start_date

dates = [str(start_date + timedelta(days=i))[:4] + '/'
                + str(start_date + timedelta(days=i))[5:7] + '/'
                + str(start_date + timedelta(days=i))[8:10] for i in range(delta.days + 1)]

In [297]:
url = 'https://www.interfax.ru/news/2022/10/05/all/page_'

def get_day_news(url):

    inter_df = DataFrame()

    for i in range(1, 10):

        response = requests.get(url + str(i), headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        items = None
        
        if items == soup.find(class_='an').find_all('a'):
            break
            
        items = soup.find(class_='an').find_all('a')

        def get_right_link(x):

            if 'www' in x:
                return x
            else:
                return 'https://interfax.ru' + x

        links = [get_right_link(item.get('href')) for item in items]
        titles = [item.find('h3').get_text().strip() for item in items]

        inter_df = concat([inter_df, DataFrame({'title': titles, 'link': links})])
        
    return inter_df

In [298]:
interfax_df = DataFrame()
url = 'https://www.interfax.ru/news/'

for date in tqdm(dates):
    interfax_df = concat([interfax_df, get_day_news(url + date + '/all/page_')])
    
interfax_df['target'] = 0
interfax_df.drop_duplicates(inplace=True)

100%|███████████████████████████████████████████| 12/12 [00:22<00:00,  1.85s/it]


In [353]:
interfax_df

,title,link,target
0,В Москве сформирован новый состав Общественной...,https://interfax.ru/russia/865203,0
1,Около 98% жителей ЛНР высказались за присоедин...,https://interfax.ru/world/865202,0
2,"МИД Польши полагает, что проблемы ""Северный по...",https://interfax.ru/world/865201,0
3,В Осло приняли решение повысить уровень безопа...,https://interfax.ru/world/865200,0
4,"Компания-оператор NS1 уверена, что утечка газа...",https://interfax.ru/world/865199,0
...,...,...,...
21,"КНДР назвала ракетные пуски защитой ""от прямых...",https://interfax.ru/world/866907,0
22,Инспекторы осмотрели более 500 судов в рамках ...,https://interfax.ru/world/866906,0
23,СМИ сообщили о намерении Байдена ограничить пр...,https://interfax.ru/world/866905,0
24,Сбербанк повысил ставки по ипотеке из-за измен...,https://interfax.ru/business/866904,0


## Бухгалтер: buh.ru

In [317]:
buh_url = 'https://buh.ru/news/uchet_nalogi/?PAGEN_1='

buh_df = DataFrame()

for i in tqdm(range(1, 50)):
    
    url = buh_url + str(i)

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    items = soup.find(class_="tab-pane fade active").find_all('a')[:-7]
    links = ['https://buh.ru' + item.get('href') for item in items]
    titles = [item.get_text().strip() for item in items]

    buh_df = concat([buh_df, DataFrame({'title': titles, 'link': links})])
    
buh_df['target'] = 1
buh_df.drop_duplicates(inplace=True)

100%|███████████████████████████████████████████| 49/49 [01:30<00:00,  1.84s/it]


In [352]:
buh_df

,title,link,target
0,"Повышение лимитов для УСН, упрощенные правила ...",https://buh.ru/news/uchet_nalogi/155283/,1
1,Единый семинар 1С для бухгалтеров и руководите...,https://buh.ru/news/uchet_nalogi/153831/,1
2,Работодателей обязали сдавать СЗВ-ТД при приос...,https://buh.ru/news/uchet_nalogi/155279/,1
3,Вправе ли работодатель отправлять форму СТД-Р ...,https://buh.ru/news/uchet_nalogi/155277/,1
4,Опубликована видеозапись лекции об использован...,https://buh.ru/news/uchet_nalogi/155275/,1
...,...,...,...
6,"ПФР внес изменения в формы СЗВ-ТД, СЗВ-СТАЖ и ...",https://buh.ru/news/uchet_nalogi/152756/,1
7,Обязательную маркировку жидкостей для электрон...,https://buh.ru/news/uchet_nalogi/152755/,1
8,"Работодателей предупредили, когда суточные под...",https://buh.ru/news/uchet_nalogi/152753/,1
9,Самое новое в «1С:Бухгалтерии 8»: заполнение о...,https://buh.ru/news/uchet_nalogi/152726/,1


## Бухгалтер: audit-it.ru

In [333]:
audit_url = 'https://www.audit-it.ru/news/account/'

audit_df = DataFrame()

for i in tqdm(range(1, 50)):
    
    response = requests.get(audit_url + str(i), headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    items = soup.find(class_='news-list').find_all('a')
    links = ['https://audit-it.ru' + item.get('href') for item in items]
    titles = [item.get_text().strip() for item in items]

    audit_df = concat([audit_df, DataFrame({'title': titles, 'link': links})])
    
audit_df['target'] = 1
audit_df.drop_duplicates(inplace=True)

100%|███████████████████████████████████████████| 49/49 [02:33<00:00,  3.14s/it]


In [355]:
audit_df

,title,link,target
0,Наиболее важные новости недели 3 - 7 октября 2...,https://audit-it.ru/news/account/1067688.html,1
1,Белоусов поддержал идею отказа от австралийско...,https://audit-it.ru/news/account/1067686.html,1
2,В кабмине поддержали идею изменения Налогового...,https://audit-it.ru/news/account/1067687.html,1
3,НДС с аванса восстанавливается в периоде отгру...,https://audit-it.ru/news/account/1067677.html,1
4,Заплатили в последний день - недоимки по взнос...,https://audit-it.ru/news/account/1067599.html,1
...,...,...,...
15,Долю участия в международной компании не учест...,https://audit-it.ru/news/account/1051126.html,1
16,"ФНС не сдаст: сведения, полученные в рамках ко...",https://audit-it.ru/news/account/1051818.html,1
17,"Контрагентам доверяй, но для ФНС проверяй",https://audit-it.ru/news/account/1052036.html,1
18,В Госдуму внесен законопроект о праве малого б...,https://audit-it.ru/news/account/1052023.html,1


## Информационная безопасность: anti-malware.ru

In [335]:
anti_url = 'https://www.anti-malware.ru/news?page='

anti_df = DataFrame()

for i in tqdm(range(1, 130)):
    
    response = requests.get(anti_url + str(i), headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    items = soup.find_all('h2')[:-2]
    links = ['https://anti-malware.ru' + item.find('a').get('href') for item in items]
    titles = [item.find('a').get_text().strip() for item in items]
    
    anti_df = concat([anti_df, DataFrame({'title': titles, 'link': links})])
    
anti_df['target'] = 2
anti_df.drop_duplicates(inplace=True)

100%|█████████████████████████████████████████| 129/129 [02:54<00:00,  1.35s/it]


In [357]:
anti_df

,title,link,target
0,Overwatch 2 не смогла нормально запуститься из...,https://anti-malware.ru/news/2022-10-06-111332...,2
1,Kaspersky запустила бесплатный учебный курс по...,https://anti-malware.ru/news/2022-10-05-111332...,2
2,"Доступен бесплатный декриптор для файлов, заши...",https://anti-malware.ru/news/2022-10-05-114534...,2
3,Android-шпион RatMilad распространяется через ...,https://anti-malware.ru/news/2022-10-05-111332...,2
4,"Вымогателю, ставшему миллионером с помощью Net...",https://anti-malware.ru/news/2022-10-05-114534...,2
...,...,...,...
5,В России вскрыли масштабную схему мошенничеств...,https://anti-malware.ru/news/2022-01-31-114534...,2
6,Натуральный шелк способен повысить надежность ...,https://anti-malware.ru/news/2022-01-31-114534...,2
7,"В ESET рассказали о киберрисках, связанных с и...",https://anti-malware.ru/news/2022-01-31-111332...,2
8,В январе 13% попыток эксплуатации Log4Shell пр...,https://anti-malware.ru/news/2022-01-31-111332...,2


## Формирование финального датасета

In [393]:
df = concat([interfax_df, buh_df, audit_df, anti_df])

df.drop_duplicates(subset='title', inplace=True)
df.drop_duplicates(subset='link', inplace=True)

df.shape

(4648, 3)

In [394]:
df['target'].value_counts()

0    1905
1    1453
2    1290
Name: target, dtype: int64

In [396]:
df.nunique()

title     4648
link      4648
target       3
dtype: int64

## Препроцессинг данных

In [397]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

df['title'] = df['title'].apply(preprocess)

In [398]:
from nltk.corpus import stopwords 
  
stop_words = set(stopwords.words('russian')) 

def delete_stop_words(text):

    filtered_tokens = []
    for token in text.split():
        if token not in stop_words:
            filtered_tokens.append(token)
    
    return ' '.join(filtered_tokens)

df['title'] = df['title'].apply(delete_stop_words)

In [403]:
# Здесь должна быть Лемматизация, я ее убрал, чтобы быстрее все сработало, но наверное лучше с ней

from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian") 
stemming = lambda text: ' '.join([stemmer.stem(word) for word in text.split()])

df['title'] = df['title'].apply(stemming)

## TF-IDF преобразование

In [415]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test , y_train, y_test = train_test_split(df['title'].values,df['target'].values,
                                                     test_size=0.2,random_state=123,stratify=df['target'].values)
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

In [443]:
X_train_vectors.shape, X_test_vectors.shape # Второе число здесь - размерность словаря

((3718, 48796), (930, 48796))

### Бустинг на TF-IDF векторах с 3 классами

In [417]:
from catboost import CatBoostClassifier

classifier = CatBoostClassifier(border_count=3)
classifier.fit(X_train_vectors, y_train)

Learning rate set to 0.084608
0:	learn: 1.0694855	total: 78.7ms	remaining: 1m 18s
1:	learn: 1.0552247	total: 147ms	remaining: 1m 13s
2:	learn: 1.0335035	total: 215ms	remaining: 1m 11s
3:	learn: 1.0216128	total: 283ms	remaining: 1m 10s
4:	learn: 1.0073469	total: 352ms	remaining: 1m 9s
5:	learn: 0.9924745	total: 419ms	remaining: 1m 9s
6:	learn: 0.9802878	total: 486ms	remaining: 1m 8s
7:	learn: 0.9684233	total: 553ms	remaining: 1m 8s
8:	learn: 0.9573145	total: 621ms	remaining: 1m 8s
9:	learn: 0.9494799	total: 687ms	remaining: 1m 8s
10:	learn: 0.9387598	total: 754ms	remaining: 1m 7s
11:	learn: 0.9293968	total: 822ms	remaining: 1m 7s
12:	learn: 0.9198140	total: 890ms	remaining: 1m 7s
13:	learn: 0.9122880	total: 958ms	remaining: 1m 7s
14:	learn: 0.9055844	total: 1.02s	remaining: 1m 7s
15:	learn: 0.8978018	total: 1.09s	remaining: 1m 7s
16:	learn: 0.8927773	total: 1.16s	remaining: 1m 7s
17:	learn: 0.8846694	total: 1.23s	remaining: 1m 7s
18:	learn: 0.8784225	total: 1.3s	remaining: 1m 6s
19:	lea

163:	learn: 0.5585578	total: 11.2s	remaining: 57.3s
164:	learn: 0.5573004	total: 11.3s	remaining: 57.3s
165:	learn: 0.5561638	total: 11.4s	remaining: 57.2s
166:	learn: 0.5549064	total: 11.4s	remaining: 57.1s
167:	learn: 0.5535664	total: 11.5s	remaining: 57s
168:	learn: 0.5526804	total: 11.6s	remaining: 57s
169:	learn: 0.5516807	total: 11.7s	remaining: 56.9s
170:	learn: 0.5509471	total: 11.7s	remaining: 56.8s
171:	learn: 0.5498403	total: 11.8s	remaining: 56.7s
172:	learn: 0.5489869	total: 11.9s	remaining: 56.7s
173:	learn: 0.5481262	total: 11.9s	remaining: 56.6s
174:	learn: 0.5474315	total: 12s	remaining: 56.5s
175:	learn: 0.5464632	total: 12.1s	remaining: 56.4s
176:	learn: 0.5455064	total: 12.1s	remaining: 56.3s
177:	learn: 0.5442687	total: 12.2s	remaining: 56.3s
178:	learn: 0.5430970	total: 12.3s	remaining: 56.2s
179:	learn: 0.5415154	total: 12.3s	remaining: 56.1s
180:	learn: 0.5403673	total: 12.4s	remaining: 56s
181:	learn: 0.5396122	total: 12.5s	remaining: 56s
182:	learn: 0.5387198	

322:	learn: 0.4559944	total: 22.3s	remaining: 46.8s
323:	learn: 0.4553494	total: 22.5s	remaining: 47s
324:	learn: 0.4551869	total: 22.6s	remaining: 46.9s
325:	learn: 0.4546107	total: 22.7s	remaining: 46.9s
326:	learn: 0.4544707	total: 22.7s	remaining: 46.8s
327:	learn: 0.4540935	total: 22.8s	remaining: 46.7s
328:	learn: 0.4538633	total: 22.9s	remaining: 46.6s
329:	learn: 0.4533712	total: 22.9s	remaining: 46.6s
330:	learn: 0.4532813	total: 23s	remaining: 46.5s
331:	learn: 0.4530992	total: 23.1s	remaining: 46.4s
332:	learn: 0.4527461	total: 23.1s	remaining: 46.4s
333:	learn: 0.4523078	total: 23.2s	remaining: 46.3s
334:	learn: 0.4520137	total: 23.3s	remaining: 46.2s
335:	learn: 0.4515765	total: 23.4s	remaining: 46.1s
336:	learn: 0.4512978	total: 23.4s	remaining: 46.1s
337:	learn: 0.4510259	total: 23.5s	remaining: 46s
338:	learn: 0.4508373	total: 23.6s	remaining: 45.9s
339:	learn: 0.4504631	total: 23.6s	remaining: 45.9s
340:	learn: 0.4501864	total: 23.7s	remaining: 45.8s
341:	learn: 0.4500

482:	learn: 0.4080559	total: 33.9s	remaining: 36.3s
483:	learn: 0.4077412	total: 34s	remaining: 36.2s
484:	learn: 0.4076657	total: 34s	remaining: 36.1s
485:	learn: 0.4075059	total: 34.1s	remaining: 36.1s
486:	learn: 0.4073389	total: 34.2s	remaining: 36s
487:	learn: 0.4070616	total: 34.2s	remaining: 35.9s
488:	learn: 0.4065330	total: 34.3s	remaining: 35.8s
489:	learn: 0.4063744	total: 34.4s	remaining: 35.8s
490:	learn: 0.4061578	total: 34.4s	remaining: 35.7s
491:	learn: 0.4060294	total: 34.5s	remaining: 35.6s
492:	learn: 0.4058049	total: 34.6s	remaining: 35.5s
493:	learn: 0.4055209	total: 34.6s	remaining: 35.5s
494:	learn: 0.4053735	total: 34.7s	remaining: 35.4s
495:	learn: 0.4049913	total: 34.8s	remaining: 35.3s
496:	learn: 0.4047916	total: 34.9s	remaining: 35.3s
497:	learn: 0.4045813	total: 34.9s	remaining: 35.2s
498:	learn: 0.4044547	total: 35s	remaining: 35.1s
499:	learn: 0.4043215	total: 35.1s	remaining: 35.1s
500:	learn: 0.4038684	total: 35.2s	remaining: 35s
501:	learn: 0.4035668	

643:	learn: 0.3759162	total: 45.8s	remaining: 25.3s
644:	learn: 0.3758229	total: 45.9s	remaining: 25.3s
645:	learn: 0.3757276	total: 46s	remaining: 25.2s
646:	learn: 0.3755084	total: 46.1s	remaining: 25.1s
647:	learn: 0.3753381	total: 46.1s	remaining: 25.1s
648:	learn: 0.3752361	total: 46.2s	remaining: 25s
649:	learn: 0.3750456	total: 46.3s	remaining: 24.9s
650:	learn: 0.3749298	total: 46.4s	remaining: 24.8s
651:	learn: 0.3747869	total: 46.4s	remaining: 24.8s
652:	learn: 0.3746483	total: 46.5s	remaining: 24.7s
653:	learn: 0.3745088	total: 46.6s	remaining: 24.6s
654:	learn: 0.3743620	total: 46.7s	remaining: 24.6s
655:	learn: 0.3743088	total: 46.7s	remaining: 24.5s
656:	learn: 0.3741149	total: 46.8s	remaining: 24.4s
657:	learn: 0.3740113	total: 46.9s	remaining: 24.4s
658:	learn: 0.3737752	total: 46.9s	remaining: 24.3s
659:	learn: 0.3736971	total: 47s	remaining: 24.2s
660:	learn: 0.3735665	total: 47.1s	remaining: 24.2s
661:	learn: 0.3734574	total: 47.2s	remaining: 24.1s
662:	learn: 0.3731

803:	learn: 0.3526139	total: 57.1s	remaining: 13.9s
804:	learn: 0.3524863	total: 57.1s	remaining: 13.8s
805:	learn: 0.3523924	total: 57.2s	remaining: 13.8s
806:	learn: 0.3523301	total: 57.3s	remaining: 13.7s
807:	learn: 0.3521195	total: 57.3s	remaining: 13.6s
808:	learn: 0.3520650	total: 57.4s	remaining: 13.6s
809:	learn: 0.3520105	total: 57.5s	remaining: 13.5s
810:	learn: 0.3518043	total: 57.5s	remaining: 13.4s
811:	learn: 0.3516538	total: 57.6s	remaining: 13.3s
812:	learn: 0.3514605	total: 57.7s	remaining: 13.3s
813:	learn: 0.3513957	total: 57.7s	remaining: 13.2s
814:	learn: 0.3513173	total: 57.8s	remaining: 13.1s
815:	learn: 0.3512622	total: 57.9s	remaining: 13s
816:	learn: 0.3511590	total: 57.9s	remaining: 13s
817:	learn: 0.3510492	total: 58s	remaining: 12.9s
818:	learn: 0.3509314	total: 58.1s	remaining: 12.8s
819:	learn: 0.3508480	total: 58.1s	remaining: 12.8s
820:	learn: 0.3507793	total: 58.2s	remaining: 12.7s
821:	learn: 0.3505433	total: 58.3s	remaining: 12.6s
822:	learn: 0.3502

964:	learn: 0.3310347	total: 1m 7s	remaining: 2.46s
965:	learn: 0.3309722	total: 1m 8s	remaining: 2.39s
966:	learn: 0.3308903	total: 1m 8s	remaining: 2.32s
967:	learn: 0.3307975	total: 1m 8s	remaining: 2.25s
968:	learn: 0.3307561	total: 1m 8s	remaining: 2.18s
969:	learn: 0.3306998	total: 1m 8s	remaining: 2.11s
970:	learn: 0.3306123	total: 1m 8s	remaining: 2.04s
971:	learn: 0.3305274	total: 1m 8s	remaining: 1.97s
972:	learn: 0.3302626	total: 1m 8s	remaining: 1.9s
973:	learn: 0.3302269	total: 1m 8s	remaining: 1.83s
974:	learn: 0.3301282	total: 1m 8s	remaining: 1.76s
975:	learn: 0.3300529	total: 1m 8s	remaining: 1.69s
976:	learn: 0.3298356	total: 1m 8s	remaining: 1.62s
977:	learn: 0.3297789	total: 1m 8s	remaining: 1.55s
978:	learn: 0.3297170	total: 1m 8s	remaining: 1.48s
979:	learn: 0.3295032	total: 1m 8s	remaining: 1.41s
980:	learn: 0.3294403	total: 1m 9s	remaining: 1.34s
981:	learn: 0.3292443	total: 1m 9s	remaining: 1.27s
982:	learn: 0.3292117	total: 1m 9s	remaining: 1.2s
983:	learn: 0.

In [432]:
y_pred = classifier.predict(X_test_vectors).squeeze()

In [441]:
y_test.shape, y_pred.shape

((930,), (930,))

In [445]:
print('Верных предсказаний модели:', np.count_nonzero(y_test == y_pred))

Верных предсказаний модели: 820


### Пример работы модели

На заголовке новости, которой не было в изначальной выборке. С сайта buh.ru

In [492]:
text = 'Минфину предложили включить в базу налога на прибыль расходы на дополнительные отпуска вакцинированным работникам'

text = preprocess(text)
text = delete_stop_words(text)
text = stemmer.stem(text)

text

'минфину предложили включить базу налога прибыль расходы дополнительные отпуска вакцинированным работник'

In [493]:
text = vectorizer.transform([text])

text

<1x48796 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [494]:
classifier.predict_proba(text) # Вероятность того, что этот текст относится к категории "Бухгалтерия" - 0.56 
                                # теперь лишь надо выбрать порог, выше которого будем действительно относить к бухгалтерии
                                # так как новостей много, то он скорее всего должен быть высоким

array([[0.33277532, 0.56337753, 0.10384715]])

## Извлечение трендов

In [585]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,3))

In [616]:
def get_trends(corpus, vectorizer):

    X = vectorizer.fit_transform(corpus['title']).toarray()
    
    count = X.shape[0] // 10
    
    result = []
        
    new_freq = np.asarray(X[:count].sum(axis=0)).ravel() / len(X[:count])
    old_freq = np.asarray(X[count:].sum(axis=0)).ravel() / len(X[count:])
    
    razn = np.array(new_freq - old_freq)
    vocab = np.array(vectorizer.get_feature_names_out())
    
    return vocab[np.argsort(razn)[-40:]]

result = get_trends(anti_df, vectorizer)
result

array(['билдер', 'установки', 'мфа', 'доступен', 'миллиона', 'телефонных',
       'режим', 'новая', 'проникли', 'сп', 'внедрили', 'используются',
       'висион', 'назвали', 'бесплатный', 'злодеи', 'сеть', 'отсрочку',
       'вызывает', 'расширенным', 'убер', 'отрасл', 'самоходный',
       'еxчанг', 'миcрософт заблокировала', 'сайты', 'роcкстар',
       'новая версия', 'используют', 'жертв', 'рcе', 'wиндоwс', '11 22h2',
       '22h2', 'wиндоwс 11 22h2', '11', 'ыоутубе', 'атак', 'хакеры',
       'wиндоwс 11'], dtype=object)

In [617]:
from natasha import Segmenter, NewsMorphTagger, Doc

segmenter = Segmenter()
morph_tagger = NewsMorphTagger(emb)

trend = []

for gram in result:
    
    doc = Doc(gram)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    synt_length = len(doc.tokens)
    
    if synt_length == 1 and doc.tokens[0].pos == 'NOUN':
        trend.append(gram)
        
    elif synt_length > 1 and (str(doc.tokens[0]).isdigit() == False) and (str(doc.tokens[1]).isdigit() == False):
        trend.append(gram)
        
np.array(trend)

array(['билдер', 'установки', 'мфа', 'миллиона', 'режим', 'сп', 'висион',
       'злодеи', 'сеть', 'отсрочку', 'убер', 'отрасл', 'еxчанг',
       'миcрософт заблокировала', 'сайты', 'роcкстар', 'новая версия',
       'жертв', 'рcе', 'wиндоwс', '11 22h2', '22h2', 'wиндоwс 11 22h2',
       'ыоутубе', 'атак', 'wиндоwс 11'], dtype='<U23')